## 数据

Caption 的数据分为 instances.json 和 annotations.json.

instances.json 的 annotations 字段中有一个 images 列表，列表的每一个元素对应一张图片的信息，其中重要的是`id`和`image_id`:
+ id: 在 instances.json 中的 annotations 字段中 images 列表中每个条目的自身 id （ id 是这个条目的一个字段，整个条目是一个字典）
+ image_id: 列表中每个条目对应的图片的 ID

经过 cocoapi 返回的是一个字典 anns ， key:id, value:image 列表中该 id 对应的整个条目

那么(样本，类标 ) 或者(image, caption) 对的匹配方法就是通过从 anns 中取出来一个元素，得到 image_id

而对于 annotations.json 里面存储的是 image_id 和 caption 的对应关系。并且一个 image 对应多个 caption 。同样是调用 cocoapi （返回的东西名字相同，数据不一样），这时候侧重的就不同了。

对于 annotations.json 侧重的就是根据输入的 image_id 返回一个 anno_id 组成的列表。

---

实际上 instances.json 还包含图像 bbox （进行 detection 图像检测）、segmentation （进行图像分割），以及对应的图像名字。而 annotations.json 只是在 Caption 中需要的文件。

## 预处理

图片的预处理 RGB 这三个 Channel 的数值都进行标准化，然后映射到[0-1] 之间。

统计所有 caption 训练数据中单词的出现次数，如果一个 caption 中的任意一个单词的出现次数小于`vocab_threshold`，则从数据集中删除这个 caption 。
所以在训练的时候是根据 caption 一条条训练（根据 caption 找对应的 image) 。

## 训练

### data_loader.dataset

类似于 Pytorch 的 DataLoader ，传入数据集和`batch_size`之后返回的变量有一个`dataset`成员一样。

```python
train_dl = DataLoader(train_ds, batch_size=bs)
```

定义一个 get_loader 函数和 CoCoDataset 类。 get_loader 返回 Pytorch DataLoader 实例（参数是 COCODataset 实例，与 Pytorch 的 DataLoader 接口一致）。

COCODataset 有一个 vocab 成员，这个成员有两个 dict 方别存放 word->id, id->word 的映射关系，也就是每个参与训练的单词有自己的 id ，包括 start_word, end_word, unknown_word. 这个 vocab 可以保存下来，为下次训练时使用（vocab 只和 vocab_threshold 有关)，以备下次网格训练时使用。

COCODataset 还会生成一个列表（ caption_lengths ），每个元素是遍历 annotation 时该 caption 的单词的个数。然后训练抽样的方法是：从 caption_lengths 中选择一个值，然后列出所有样本中包含的 wrod 个数都为这个值的样本。在从这些样本中随即选择 batch_size 个进行训练。

之所以这样是因为如此才能对 NLP 进行批训练。不然每个样本的长度不一样，就只能一条一条的 caption 进行训练（因为不同长度 Pytorch 生成的动态图不一样）。无法享受 GPU 的加速。

COCODataset 实现了 \_\_getitem\_\_ 接口接收一个 annotation 索引，然后返回这个 annotation 对应的进过预处理的图片和经过分词得到的单词对应的 id 组成的 list.

然后就把图片送入到 EncoderCNN 中来抽取信息，然后通过 DecoderRNN 来生成 caption. 而训练就是通过计算生成的 caption 和 类标的损失来学习参数。

### EncoderCNN

图像特征的抽取网络使用的resnet50，去掉了最后的全连接层。抽取的特征维度为$(1, 2048, 1, 1)$.


## 考点

### ResNet 为什么好？

### 为什么设置 vocab_threshold？

偏差和方差的问题，不设置就容易过拟合。


## 数字图像处理关注的问题

<img src="images/image-process.jpg" alt="drawing" width="400" height="300"/>

1. 分类：这张图像中有气球。
2. 语义分割：这些像素全是气球像素。
3. 目标检测：这张图像中的这些位置上每个位置有一个气球（然后可以统计出一共有$7$个气球）。
4. 实例分割：这些位置上有$7$个（统计出来的）气球，并且这些像素分别属于每个气球。


## 图片预处理归一化的作用

1. 消除图像的共性，凸显个体差异
    其实也不只是图像，所有样本都是。比如下面的第一个图是样本在两个特征的分布：

    <img src="images/two-features.jpg" alt="drawing" width="400" height="250"/>

    而第二张图更加的鲜艳，更加突出了这张图中那些位置存在边缘。

    <img src="images/scale-flower.jpg" alt="drawing" width="400" height="250"/>

2. 归一化与机器学习样本的区别
    图像的值在[0-255] ，方差基本一样。其实，只做去均值处理就可以了。

3. 梯度下降学习速度
    对于机器学习数据，会被量级大的特征 dominant ，训练过程容易震荡。
    而对于图像，当使用 relu 激活函数的时候，因为图像的输入值不做均值化都是正值，那么输出值也是正值，这样同一轮学习中所有参数的梯度根据损失函数也只能全是正或者全是负。而通常我们希望的是或者对于一个参数的学习路线是 Z 字形，可以理解为正样例学习一下，负样例学习一下（也可以是奇数轮学习和偶数轮学习）。

    <img src="images/zag-gradient.jpg" alt="drawing" width="400" height="250"/>

## refs

1. [MS-COCO 数据集的内容说明、数据的定义、标注信息 - 简书 ](https://www.jianshu.com/p/568a2f5195a9)
2. [答案解析(1)—史上最全 Transformer 面试题：灵魂 20 问帮你彻底搞定 Transformer - 知乎 ](https://zhuanlan.zhihu.com/p/149799951)
3. [深度学习中，图片的预处理为什么要减去图片的平均值，在什么情况下要选择这种预处理方式？ - blateyang 的回答 - 知乎 ](https://www.zhihu.com/question/49096923/answer/518032757)
4. [深度学习中，图片的预处理为什么要减去图片的平均值，在什么情况下要选择这种预处理方式？ - 徐之谓的回答 - 知乎 ](https://www.zhihu.com/question/49096923/answer/743038532)